In [ ]:
import pandas as pd
import re
import yaml
import os

In [ ]:
job_files = []

for root, dirs, files in os.walk(r"D:\Git\example_infrastructure_data_dev\modules\data_lake\local"):
    for file in files:
        if file.endswith(".hcl"):
            print(os.path.join(root, file))
            job_files.append(os.path.join(root, file))

In [ ]:

jobs_info_list = []


keywords = ["job","type","cron","group","task","driver","command","args"]

for job in job_files[:]:
    with open(job, "r") as f:
        data = f.readlines()
        f.close()

    job_info_dict = {}
        

    for line in data:
        for keyword in keywords:
            result = re.search(f"[\s]?{keyword}\s",line)
            if result:
                try:
                    # print(result)
                    job_info_dict[keyword] = re.search(r'[\s\[]?\"([^\"]+)\"[\n\s]?', line).group(1)
                except:
                    print(line)
    job_info_dict["job_file_path"] = re.sub(r"D:\\Git\\example_infrastructure_data_dev\\modules", "", job)
    jobs_info_list.append(job_info_dict)
            
    
            
# print(job_info_dict)


In [ ]:
df = pd.DataFrame(jobs_info_list)

In [ ]:
mask = df["job"].str.contains("acronis")

In [ ]:
df.loc[mask, "cron"] = "0 * * * *"

In [ ]:
df[mask]

In [ ]:
df

In [ ]:
df.to_csv(".csv", index=False)

In [ ]:

def return_hcl_job(row):

    template = f""" 
    job "{row["job"]}" {{
    
      type = "{row["type"]}"
    
      periodic {{
        cron             = "{row["cron"]}"
        prohibit_overlap = true
      }}
    
      group "{row["group"]}" {{
        count = 1
        task "{row["task"]}" {{
          driver = "{row["driver"]}"
    
          config {{
            command = "{row["command"]}"
            args    = ["{row["args"]}"]
          }}
    
          dispatch_payload {{
            file = ""
          }}
    
        }}
      }}
    }}    
    """
    
    return template

In [ ]:
job_files_loc = r"D:\Git\example_infrastructure_data_dev\projects\2023\nomad_configuration_standards\job_files"

In [ ]:
for index, row in df.iterrows():
    print(row["job"])
    dest_loc = f'{job_files_loc}/{row["job"]}.hcl'
    with open(dest_loc, "w") as f:
        f.write(return_hcl_job(row))
        f.close()